In [1]:
# Import all the things

import json
import pandas as pd
import numpy as np
import wrangle
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords, split_data, prep_string_data

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = wrangle.get_npr_data()
df.head()

The df has 2998347 rows and 11 columns.


,episode_id,episode_order,speaker,utterance,program,title,episode_date,is_host,clean,lemmatized_stopped,vader_stopped
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ...",Talk of the Nation,How To Watch The Tour De France,2010-07-12,False,it s a 2200 mile race. to give some sense of p...,2200 mile race. give sense perspective roughly...,0.0000
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...,Talk of the Nation,How To Watch The Tour De France,2010-07-12,False,so for a top competitor like lance to try to m...,top competitor like lance try make much time 1...,0.9313
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,...",Talk of the Nation,How To Watch The Tour De France,2010-07-12,True,so in every team presumably there s one star o...,every team presumably one star one equivalent ...,0.7096
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...,Talk of the Nation,How To Watch The Tour De France,2010-07-12,False,that s right. each team has nine riders. and w...,right. team ha nine riders. basically team lea...,0.9274
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra...",Talk of the Nation,How To Watch The Tour De France,2010-07-12,True,so slipstream this is like drafting in car rac...,slipstream like drafting car racing right ?,0.3612


## Is there a difference in the mean sentiment by speaker? Program? etc
- Applied statistics-> i.e. stats testing. Is there a difference in the mean sentiment by speaker? Program? etc 

In [4]:
df.columns

Index(['episode_id', 'episode_order', 'speaker', 'utterance', 'program',
       'title', 'episode_date', 'is_host', 'clean', 'lemmatized_stopped',
       'vader_stopped'],
      dtype='object')

In [7]:
df.shape

(2998347, 11)

In [8]:
df.dtypes

episode_id              int64
episode_order           int64
speaker                object
utterance              object
program                object
title                  object
episode_date           object
is_host                  bool
clean                  object
lemmatized_stopped     object
vader_stopped         float64
dtype: object

In [9]:
df.episode_date = pd.to_datetime(df.episode_date)

In [10]:
df.dtypes

episode_id                     int64
episode_order                  int64
speaker                       object
utterance                     object
program                       object
title                         object
episode_date          datetime64[ns]
is_host                         bool
clean                         object
lemmatized_stopped            object
vader_stopped                float64
dtype: object

In [5]:
df.episode_id.value_counts()

35108    512
73336    475
57481    459
19633    453
77896    419
        ... 
85857      1
54086      1
77677      1
7000       1
31142      1
Name: episode_id, Length: 104896, dtype: int64

In [6]:
df.groupby(by='episode_id').vader_stopped.mean()

episode_id
1         0.266400
2         0.352819
3        -0.219529
4         0.059670
5         0.216645
            ...   
140746    0.265561
140747    0.068450
140748    0.330058
140749    0.163007
141179    0.422604
Name: vader_stopped, Length: 104896, dtype: float64

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns

### Looking to do a lineplot of mean sentiment by day (or maybe resampled by month or whatever), and hue on program (and later, on year...or etc)

In [27]:
sample = df.set_index('episode_date').sample(100000)
sample=sample.dropna()

In [28]:
sample.isna().sum()

episode_id            0
episode_order         0
speaker               0
utterance             0
program               0
title                 0
is_host               0
clean                 0
lemmatized_stopped    0
vader_stopped         0
dtype: int64

In [38]:
sample.resample('M').vader_stopped.mean()

episode_date
2004-01-31   -0.242350
2004-02-29         NaN
2004-03-31         NaN
2004-04-30         NaN
2004-05-31         NaN
                ...   
2019-06-30    0.183676
2019-07-31    0.191577
2019-08-31    0.168183
2019-09-30    0.207131
2019-10-31    0.156653
Freq: M, Name: vader_stopped, Length: 190, dtype: float64

In [33]:
plt.figure(figsize=(20,10))
sns.lineplot(data=drawing, x=drawing.index,y='vader_stopped', hue='program')


ValueError: Could not interpret value `vader_stopped` for parameter `y`

<Figure size 1440x720 with 0 Axes>